In [2]:
import pandas as pd

In [3]:
from MatrixExamples import m

In [11]:
class Matrix(pd.DataFrame):
    '''A Matrix in Transport Planning is a pandas DataFrame,
    with Origins and Destinations as MultiIndex levels: [O, D]'''

    @property
    def _constructor(self):
        return Matrix
    
    @property
    def Os(self):
        '''returns origin names without duplicates'''
        return self.index.get_level_values(0).unique()
    
    @property
    def Ds(self):
        '''returns destination names without duplicates'''
        return self.index.get_level_values(1).unique()
    
    @property
    def TO(self):
        '''returns trip-ends for origins'''
        return self.groupby(level=0).sum()
    
    @property
    def TD(self):
        '''returns trip-ends for destinations'''
        return self.groupby(level=1).sum()
    
    def TransposeOD(self, sort=True):
        '''Transposes a matrix in ODT format: swaps Origins and Destinations'''
        mat_T = self.swaplevel()
        mat_T.index.names = self.index.names
        if sort:
            mat_T = mat_T.sort_index()
        return mat_T
    
    @property
    def TOp(self):
        '''returns origin proportions: Pij = Tij / TOi'''
        return self.groupby(level=0).apply(lambda x: (x/x.sum()).fillna(0))
    
    @property
    def TDp(self):
        '''returns destination proportions: Pij = Tij / TDj'''
        return self.groupby(level=1).apply(lambda x: (x/x.sum()).fillna(0))
    
    @property
    def matrix(self):
        '''returns matrix as a tradicional 2D matrix'''
        return self.to_panel()

In [12]:
mat = Matrix(m)
mat

T1  T2  T3
O D            
1 1   0   1   0
  2   1   2   6
  3   1   3  10
  4   0   4  12
  5   1   5  12
  6   1   6  10
  7   0   7   6
2 1   1   8   0
  2   1   9   0
  3   0  10  10
  4   1  11  12
  5   1  12  12
  6   0  13  10
  7   1  14   6
3 1   1  15   0
  2   0  16   6
  3   1  17   0
  4   1  18  12
  5   0  19  12
  6   1  20  10
  7   1  21   6
4 1   0  22   0
  2   1  23   6
  3   1  24  10
  4   0  25   0
  5   1  26  12
  6   1  27  10
  7   0  28   6
5 1   1  29   0
  2   1  30   6
  3   0  31  10
  4   1  32  12
  5   1  33   0
  6   0  34  10
  7   1  35   6
6 1   1  36   0
  2   0  37   6
  3   1  38  10
  4   1  39  12
  5   0  40  12
  6   1  41   0
  7   1  42   6
7 1   0  43   0
  2   1  44   6
  3   1  45  10
  4   0  46  12
  5   1  47  12
  6   1  48  10
  7   0  49   0

In [13]:
mat.matrix.T3

D,1,2,3,4,5,6,7
O,,,,,,,
1,0,6,10,12,12,10,6
2,0,0,10,12,12,10,6
3,0,6,0,12,12,10,6
4,0,6,10,0,12,10,6
5,0,6,10,12,0,10,6
6,0,6,10,12,12,0,6
7,0,6,10,12,12,10,0


In [14]:
mat.TDp.matrix.T3

D,1,2,3,4,5,6,7
O,,,,,,,
1,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667
2,0.0,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667
3,0.0,0.166667,0.000000,0.166667,0.166667,0.166667,0.166667
4,0.0,0.166667,0.166667,0.000000,0.166667,0.166667,0.166667
5,0.0,0.166667,0.166667,0.166667,0.000000,0.166667,0.166667
6,0.0,0.166667,0.166667,0.166667,0.166667,0.000000,0.166667
7,0.0,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000
